# Jakarta

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

### Low 100M – 500M

In [2]:
#main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?page=1"
main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=500000000&minPrice=100000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [3]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [4]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

132

In [5]:
dataframe_low = {}
dataframe_low["Nama"] = []
dataframe_low["Harga"] = []

In [6]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=500000000&minPrice=100000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_low["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_low["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link, harga)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_low.keys():
                dataframe_low[prop_key] = []
            
            if prop_key in dataframe_low.keys():
                dataframe_low[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


/properti/jakarta-timur/hos16834646/ <span class="text-primary font-bold whitespace-pre md:text-2xl md:leading-6 text-lg leading-[16px]">Rp 422 Juta</span>
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 20 m²
Luas Bangunan: 31 m²
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 2
Kondisi Properti: Bagus

/properti/jakarta-timur/hos18641148/ <span class="text-primary font-bold whitespace-pre md:text-2xl md:leading-6 text-lg leading-[16px]">Rp 457 Juta</span>
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 21 m²
Luas Bangunan: 41 m²
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2
Kondisi Properti: Bagus

/properti/jakarta-pusat/hos18645937/ <span class="w-1 h-full bg-errorAlert rounded-tl rounded-bl"> </span>
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 21 m²
Luas Bangunan: 41 m²
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2
Kondisi Properti: Bagus

/properti/jakarta-timur/hos19481241/ <span class="text-primary font-bold whitespace-pre md:text-2xl md:leading-6 text-lg leading-[

In [9]:
rumah123_df_low = pd.DataFrame.from_dict(dataframe_low, orient='index').T
rumah123_df_low = rumah123_df_low.fillna(0)
#rumah123_df_low['Category'] = 'Low'
rumah123_df_low.to_csv('Jakarta_low.csv', index=False)

In [10]:
rumah123_df_low

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Carport,Garasi,Kamar Tidur Pembantu,Kamar Mandi Pembantu
0,rumah cantik modern minimalis lokasi strategis...,Rp 422 Juta,2,2,20 m²,31 m²,SHM,Lainnya,2,Bagus,1,1,1,1
1,Rumah 2 Lantai Murah Dan Strategis Dekat Jalan...,Rp 457 Juta,2,2,21 m²,41 m²,SHM,2200 Watt,2,Bagus,1,1,1,1
2,Rumah Murah 2 Lantai Dekat Mangga Besar Shm,,2,2,21 m²,41 m²,SHM,2200 Watt,2,Bagus,1,1,1,1
3,Rumah 2 Lantai Strategis Dekat Stasiun Jatineg...,Rp 278 Juta,1,1,15 m²,29 m²,SHM,Lainnya,2,Bagus,1,1,1,1
4,3 Lantai Rumah Akses Motor Dekat Office Salemba,Rp 470 Juta,3,2,25 m²,75 m²,SHM,2200 Watt,3,Baru,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Harga Promo Dijual Rumah 3 Kamar Dekat Kampus ...,Rp 399 Juta,2,1,60 m²,24 m²,SHM,1300 Watt,0,0,0,0,0,0
273,Soft Launching Dijua Rumah 3 Kamar Dekat Tol L...,Rp 399 Juta,0,1,60 m²,36 m²,SHM,0,0,0,0,0,0,0
274,Rumah Ready Stock Dekat Stasiun Cukup Booking ...,Rp 295 Juta,0,0,66 m²,36 m²,SHM,0,0,0,0,0,0,0
275,Murah Kpr Ready Stok Siap Huni Dekat Stasiun D...,Rp 425 Juta,0,0,50 m²,40 m²,SHM,0,0,0,0,0,0,0


In [8]:
dataframe_low

{'Nama': ['rumah cantik modern minimalis lokasi strategis di cipinang elok jakarta timur',
  'Rumah 2 Lantai Murah Dan Strategis Dekat Jalan Utama Pramuka',
  'Rumah Murah 2 Lantai Dekat Mangga Besar Shm',
  'Rumah 2 Lantai Strategis Dekat Stasiun Jatinegara Jakarta Timur',
  '3 Lantai Rumah Akses Motor Dekat Office Salemba',
  'Rumah Murah  2 Lantai Di Sunter Jakarta Utara Lokasi Strategis Bebas Banjir3',
  'Di  rumah cantik minimalis 2 lantai jalan sunter jakarta utara',
  'Rumah Murah 2 Lantai Shm Dekat Sunter Agung',
  'Rumah Murah Dan Shm Dekat Apart Basura Cipinang Jakarta Timiur',
  'Rumah Murah 2 Lantai Shm Dekat Rsud Matraman.jakarta Timur',
  'Rumah Murah Di Kawasan Cempaka Putih.dekat Kuliner Tamansolo',
  'Rumah 2 Lantai dekat Pasar Pramuka Jakarta Timur',
  'Rumah Murah 2 Lantai Ready 5 Menit Dari Stadion Jis',
  'Rumah Cantik Murah 2 Lantai 5 Menit Dr Stasiun Kereta Rajawali',
  'Rumah 2lantai jln industri 2 minimalis siap untuk pengantin baru',
  'Rumah DP Murah Ada Carp

### Lower-Mid	500M – 1B

In [3]:
main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=1000000000&minPrice=500000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [4]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [5]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

419

In [6]:
dataframe_low_mid = {}
dataframe_low_mid["Nama"] = []
dataframe_low_mid["Harga"] = []

In [7]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=1000000000&minPrice=500000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_low_mid["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_low_mid["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_low_mid.keys():
                dataframe_low_mid[prop_key] = []
            
            if prop_key in dataframe_low_mid.keys():
                dataframe_low_mid[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/jakarta-pusat/hos19262022/ Rp 780 Juta
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 24 m²
Luas Bangunan: 71 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 3
Kondisi Properti: Bagus

1
/properti/jakarta-timur/hos18686806/ Rp 940 Juta
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 106 m²
Luas Bangunan: 70 m²
Sertifikat: SHM
Daya Listrik: 1300 Watt
Kamar Tidur Pembantu: 1
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/jakarta-timur/hos18115790/  
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 92 m²
Luas Bangunan: 78 m²
Sertifikat: SHM
Daya Listrik: 4400 Watt
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/jakarta-utara/hos18020362/ Rp 840 Juta
Kamar Tidur: 2
Kamar Mandi: 2
Luas Tanah: 32 m²
Luas Bangunan: 53 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2

1
/properti/jakarta-timur/hos18881982/ Rp 550 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 76 m²
Luas Bangunan: 53 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Garasi: 

In [22]:
df_low_mid = pd.DataFrame.from_dict(dataframe_low_mid, orient='index').T
df_low_mid = df_low_mid.fillna(0)
df_low_mid['Category'] = 'Low-Mid'
df_low_mid.to_csv('Jakarta_low_mid.csv', index=False)

In [23]:
df_low_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Kamar Tidur Pembantu,Garasi,Kamar Mandi Pembantu,Category
0,Rimah Dengan Lokasi Strategis Di Kepu Kemayora...,Rp 780 Juta,2,2,24 m²,71 m²,1,SHM,2200 Watt,3,Bagus,1,1,2,Low-Mid
1,Rumah Luas Strategis 10 Menit ke Trans Studio ...,Rp 940 Juta,2,2,106 m²,70 m²,1,SHM,1300 Watt,1,Bagus,3,1,1,Low-Mid
2,Rumah Bebas Banjir 15 Menit ke Stasiun LRT Har...,,2,1,92 m²,78 m²,1,SHM,4400 Watt,1,Bagus,1,1,1,Low-Mid
3,Rumah Baru 2 Lantai Cluster One Gate Tanjung P...,Rp 840 Juta,2,2,32 m²,53 m²,1,SHM,2200 Watt,2,Baru,1,1,1,Low-Mid
4,Booking 10 Jt All In Dekat Kota Wisata Akses L...,Rp 550 Juta,2,1,76 m²,53 m²,1,SHM,1300 Watt,2,Baru,1,1,1,Low-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,"Beli Sekarang Juga, Promo Awal Tahun Dengan 2 ...",Rp 880 Juta,2,1,90 m²,75 m²,0,SHM,Lainnya,0,0,0,0,0,Low-Mid
212,Dijual Cepat Rumah Di Bukit Sentul.cluster Tam...,,0,0,38 m²,120 m²,0,SHM,Lainnya,0,0,0,0,0,Low-Mid
213,Rumah Siap Huni Di Jagakarsa Jakarta Selatan D...,Rp 900 Juta,0,0,90 m²,80 m²,0,SHM,2200 Watt,0,0,0,0,0,Low-Mid
214,"Legenda wisata , Rumah Minimalis 1 Lantai - Bi...",Rp 885 Juta,0,0,67 m²,70 m²,0,SHM,Lainnya,0,0,0,0,0,Low-Mid


### Upper-Mid	1B – 3B

In [6]:
main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=3000000000&minPrice=1000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [7]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [8]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [9]:
dataframe_up_mid = {}
dataframe_up_mid["Nama"] = []
dataframe_up_mid["Harga"] = []

In [11]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=3000000000&minPrice=1000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_up_mid["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_up_mid["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_up_mid.keys():
                dataframe_up_mid[prop_key] = []
            
            if prop_key in dataframe_up_mid.keys():
                dataframe_up_mid[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/jakarta-barat/hos19592584/ Rp 2,17 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 90 m²
Luas Bangunan: 120 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/jakarta-barat/hos19592480/ Rp 2,05 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 105 m²
Luas Bangunan: 120 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/jakarta-selatan/hos19482702/ Rp 1,65 Miliar
Kamar Tidur: 8
Kamar Mandi: 4
Luas Tanah: 178 m²
Luas Bangunan: 150 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Renovasi Total

1
/properti/jakarta-barat/hos19524790/ Rp 2,3 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 90 m²
Luas Bangunan: 120 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/jakarta-selatan/hos19429218/  
Kamar

In [15]:
df_up_mid = pd.DataFrame.from_dict(dataframe_up_mid, orient='index').T
df_up_mid = df_up_mid.fillna(0)
df_up_mid['Category'] = 'Up-Mid'
df_up_mid.to_csv('Jakarta_up_mid.csv', index=False)

In [16]:
df_up_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Garasi,Jumlah Lantai,Kondisi Properti,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,6 X 15 Mewah Termurah Di Metland,"Rp 2,17 Miliar",3,2,90 m²,120 m²,2,SHM,Lainnya,1,2,Bagus,1,1,Up-Mid
1,6 X 15 Mewah Termurah Di Metland,"Rp 2,17 Miliar",3,2,105 m²,120 m²,2,SHM,Lainnya,1,2,Bagus,1,1,Up-Mid
2,7 X 15 Termurah Di Metland Blok E Bagus,"Rp 2,05 Miliar",8,4,178 m²,150 m²,1,SHM,2200 Watt,1,2,Renovasi Total,1,1,Up-Mid
3,Rumah 2 Lantai Di Bilangan Jl. Rc Veteran Bint...,"Rp 1,65 Miliar",3,2,90 m²,120 m²,1,SHM,Lainnya,1,2,Bagus,1,1,Up-Mid
4,Dijual Rumah Di Citra 3 Siap Huni Bebas Banjir,"Rp 2,3 Miliar",3,3,70 m²,80 m²,1,SHM,2200 Watt,2,2,Bagus,1,1,Up-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Rumah Minimalis di Taman Aries Kebon Jeruk,"Rp 1,99 Miliar",0,3,60 m²,77 m²,0,HGB,Lainnya,0,0,0,0,0,Up-Mid
358,Rumah Minimalis di Taman Aries Kebon Jeruk,"Rp 1,99 Miliar",0,0,122 m²,250 m²,0,SHM,2200 Watt,0,0,0,0,0,Up-Mid
359,Dijual Rumah Dekat Kelapa Gading One Gate Syst...,"Rp 2,7 Miliar",0,0,105 m²,105 m²,0,SHM,Lainnya,0,0,0,0,0,Up-Mid
360,Dijual Rumah Dekat Kelapa Gading Kondisi Bagus...,"Rp 2,8 Miliar",0,0,120 m²,180 m²,0,HGB,2200 Watt,0,0,0,0,0,Up-Mid


### High 3B-7B

In [26]:
main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=7000000000&minPrice=3000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [27]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [28]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [29]:
dataframe_high = {}
dataframe_high["Nama"] = []
dataframe_high["Harga"] = []

In [30]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=7000000000&minPrice=3000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_high["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_high["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_high.keys():
                dataframe_high[prop_key] = []
            
            if prop_key in dataframe_high.keys():
                dataframe_high[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


2
/properti/jakarta-timur/hos19219550/ Rp 4,7 Miliar
Kamar Tidur: 5
Kamar Mandi: 4
Luas Tanah: 190 m²
Luas Bangunan: 235 m²
Carport: 3
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 2
Jumlah Lantai: 2
Kondisi Properti: Bagus

2
/properti/jakarta-timur/hos16711057/ Rp 3,5 Miliar
Kamar Tidur: 4
Kamar Mandi: 4
Luas Tanah: 200 m²
Luas Bangunan: 200 m²
Carport: 1
Sertifikat: Lainnya
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 4
Kondisi Properti: Bagus

2
/properti/jakarta-barat/hos18706923/  
Kamar Tidur: 3
Kamar Mandi: 3
Luas Tanah: 180 m²
Luas Bangunan: 250 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 7700 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

2
/properti/jakarta-timur/hos14929295/ Rp 3,3 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 105 m²
Luas Bangunan: 108 m²
Carport: 2
Sertifikat: PPJB
Daya Listrik: 3500 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

2
/properti/jakar

In [35]:
df_high = pd.DataFrame.from_dict(dataframe_high, orient='index').T
df_high = df_high.fillna(0)
df_high['Category'] = 'High'
df_high.to_csv('Jakarta_high.csv', index=False)

In [36]:
df_high

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Garasi,Jumlah Lantai,Kondisi Properti,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,Dijual Cepat Rumah Siap Huni Dan Posisi Hoek D...,"Rp 4,7 Miliar",5,4,190 m²,235 m²,3,SHM,Lainnya,2,2,Bagus,1,1,High
1,Rumah Cantik Siap Huni di Raffles Hills Cibubur,"Rp 3,5 Miliar",4,4,200 m²,200 m²,1,Lainnya,Lainnya,1,4,Bagus,1,1,High
2,Rumah 2 Lantai Taman Villa Meruya 9x20 Banguna...,,3,3,180 m²,250 m²,1,SHM,7700 Watt,1,2,Bagus,1,1,High
3,Rumah Mewah Lokasi Selangkah Ke Aeon Mall .ike...,"Rp 3,3 Miliar",3,2,105 m²,108 m²,2,PPJB,3500 Watt,1,2,Bagus,1,1,High
4,Dijual Murah Turun Harga Rumah Tebet jalan 2 m...,"Rp 5,5 Miliar",5,3,217 m²,320 m²,2,SHM,Lainnya,2,2,Bagus,1,1,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,"Rumah 2 Lantai Bagus SHM di Cibubur, Jakarta T...","Rp 6,2 Miliar",0,0,300 m²,255 m²,0,SHM,3500 Watt,0,0,0,0,0,High
244,Dijual Rumah Baru Renov Total di Second Boulev...,"Rp 6,8 Miliar",0,0,300 m²,450 m²,0,Lainnya,Lainnya,0,0,0,0,0,High
245,Di Jual Rumah Hook Sudah Renovasi Hadap Timur ...,"Rp 5,25 Miliar",0,0,394 m²,500 m²,0,SHM,4400 Watt,0,0,0,0,0,High
246,Rumah Pik Mediterania Boulevard Kuda Laut 7x20,"Rp 5,5 Miliar",0,0,140 m²,300 m²,0,HGB,2200 Watt,0,0,0,0,0,High


### Luxury 7B-20B

In [2]:
main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=20000000000&minPrice=7000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [3]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [4]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [5]:
dataframe_luxury = {}
dataframe_luxury["Nama"] = []
dataframe_luxury["Harga"] = []

In [6]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?maxPrice=20000000000&minPrice=7000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_luxury["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_luxury["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_luxury.keys():
                dataframe_luxury[prop_key] = []
            
            if prop_key in dataframe_luxury.keys():
                dataframe_luxury[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/jakarta-selatan/hos3659326/ Rp 8,45 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 433 m²
Luas Bangunan: 250 m²
Sertifikat: SHM
Daya Listrik: 4400 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 4
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/jakarta-selatan/hos15075938/  
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 327 m²
Luas Bangunan: 146 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 4400 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/jakarta-selatan/hos13993246/ Rp 7,25 Miliar
Kamar Tidur: 3
Kamar Mandi: 3
Luas Tanah: 216 m²
Luas Bangunan: 375 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/jakarta-selatan/hos38838412/ Rp 17,5 Miliar
Kamar Tidur: 6
Kamar Mandi: 5
Luas Tanah: 685 m²
Luas Bangunan: 481 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 23000 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 2
Jumlah Lantai:

In [8]:
df_luxury = pd.DataFrame.from_dict(dataframe_luxury, orient='index').T
df_luxury = df_luxury.fillna(0)
df_luxury['Category'] = 'Luxury'
df_luxury.to_csv('Jakarta_luxury.csv', index=False)

In [9]:
df_luxury

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Jumlah Lantai,Kondisi Properti,Carport,Category
0,"Rumah dalam Kompleks Buncit Indah, Jakarta Sel...","Rp 8,45 Miliar",3,2,433 m²,250 m²,SHM,4400 Watt,1,1,4,1,Bagus,2,Luxury
1,"Dijual Rumah di Komplek Garuda Pancoran, Jakar...",,3,2,327 m²,146 m²,SHM,4400 Watt,1,1,1,2,Bagus,2,Luxury
2,"Rumah di Antasari Residence, Jakarta Selatan","Rp 7,25 Miliar",3,3,216 m²,375 m²,SHM,Lainnya,1,1,2,1,Bagus,2,Luxury
3,RUMAH MEWAH EXCLUSIVE MODERN CANTIK DAN TERSED...,"Rp 17,5 Miliar",6,5,685 m²,481 m²,SHM,23000 Watt,1,1,2,2,Bagus,2,Luxury
4,Rumah Di Taman Kebon Jeruk Intercon Jakarta Ba...,"Rp 9,5 Miliar",4,4,450 m²,600 m²,SHM,6600 Watt,1,1,2,2,Renovasi Minimum,2,Luxury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Dijual Rumah Modern Minimalist Style Yang Tera...,Rp 17 Miliar,0,0,397 m²,350 m²,SHM,Lainnya,0,0,0,0,0,0,Luxury
213,"Dijual Rumah 2 Lantai Area Lebak Bulus Raya, ...","Rp 9,6 Miliar",0,0,440 m²,400 m²,SHM,Lainnya,0,0,0,0,0,0,Luxury
214,Rumah Strategis Pinggir Jalan & Cocok Untuk Ru...,Rp 16 Miliar,0,0,488 m²,210 m²,SHM,Lainnya,0,0,0,0,0,0,Luxury
215,RUMAH JALAN CIOMAS KEBAYORAN BARU JAKARTA SEL...,Rp 12 Miliar,0,0,207 m²,180 m²,SHM,Lainnya,0,0,0,0,0,0,Luxury


In [5]:
rumah123_df_low.shape

NameError: name 'rumah123_df_low' is not defined

In [13]:
df_low_mid

NameError: name 'df_low_mid' is not defined